# 依據pair_trading中的每個.py檔檢查

In [1]:
import pandas as pd
from datetime import datetime
from pair_trading.basic_tool import *

In [2]:
data = pd.read_csv('data/data.csv')
data.date = data.date.apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))

In [3]:
stock_2330 = data[data.id == 2330]
stock_2330.reset_index(inplace=True)
stock_2330 = stock_2330.copy()

# basic_tool.py

## crossover

In [4]:
stock_2330.loc[:, 'ma5'] = stock_2330.close.rolling(5).mean()
crossover_True = crossover(stock_2330.close, stock_2330.ma5)

In [5]:
crossover_True[crossover_True == True].head()

12    True
30    True
39    True
53    True
60    True
dtype: bool

In [6]:
stock_2330.iloc[11:13]

,index,id,name,date,open,high,low,close,volume,foreign,trust,dealer,ma5
11,44774,2330,台積電,2000-12-27,18.97,18.97,18.60,18.72,20502,11603,-1270,-345,19.444
12,44784,2330,台積電,2000-12-28,19.22,19.97,18.85,19.97,35549,7238,1620,179,19.370


In [7]:
# 再找一個例子
crossover_True[crossover_True == True].tail()

4450    True
4456    True
4462    True
4464    True
4469    True
dtype: bool

In [8]:
stock_2330.iloc[4468:4470]

,index,id,name,date,open,high,low,close,volume,foreign,trust,dealer,ma5
4468,201104,2330,台積電,2018-12-26,218.5,219.5,216.0,216.5,16893,-41,-331,-946,219.8
4469,201154,2330,台積電,2018-12-27,222.0,224.0,220.5,223.0,21915,8967,-34,24,219.7


## crossunder

In [9]:
crossunder_True = crossunder(stock_2330.close, stock_2330.ma5)

In [10]:
crossunder_True[crossunder_True == True].head()

28    True
32    True
43    True
56    True
62    True
dtype: bool

In [11]:
stock_2330.iloc[27:29]

,index,id,name,date,open,high,low,close,volume,foreign,trust,dealer,ma5
27,44934,2330,台積電,2001-01-18,25.71,25.84,25.21,25.46,66857,33564,-647,-1568,24.588
28,44944,2330,台積電,2001-01-29,24.96,25.34,24.21,24.21,46543,7706,-1243,-1696,24.736


In [12]:
# 再找一個例子
crossunder_True[crossunder_True == True].tail()

4452    True
4459    True
4463    True
4465    True
4471    True
dtype: bool

In [13]:
stock_2330.iloc[4462:4464]

,index,id,name,date,open,high,low,close,volume,foreign,trust,dealer,ma5
4462,89280,2330,台積電,2018-12-19,222.0,225.5,222.0,225.5,23415,3156,271,-492,224.0
4463,89290,2330,台積電,2018-12-20,221.0,222.5,221.0,221.0,38697,-10972,166,360,223.0


# strategy.py

In [14]:
# 以eaxmple資料夾中的example為例子
from pair_trading.strategy import Strategy

# data
stock_2330 = data[data.id == 2330]
stock_1101 = data[data.id == 1101]
stock_2330, stock_1101 = preprocess(stock_2330, stock_1101)

# 計算指標
stock_2330['ma20'] = stock_2330.close.rolling(20).mean()
stock_2330['ma5'] = stock_2330.close.rolling(5).mean()
stock_1101['ma20'] = stock_1101.close.rolling(20).mean()
stock_1101['ma5'] = stock_1101.close.rolling(5).mean()

# 撰寫進出場條件
# (條件為list，若依序填入則為條件皆符合才觸發條件("且"))
# 進場訊號
condition_in = [
    # 此為"且"的寫法
    # 台積電今日收盤價高過於前十日最高價
    stock_2330.close > lag(maximum(stock_2330.high, 10)),
    # "且"今日收紅k
    stock_2330.close > stock_2330.open
]

# 出場訊號
condition_out = [
    # 此為"或"的寫法
    (
        # 台積電20日平均死亡交叉5日平均(類似Multicharts語法)
        (crossunder(stock_2330.ma20, stock_2330.ma5)) |
        # "或"今日收黑小於0.03%的黑k
        ((stock_2330.close - stock_2330.open)/stock_2330.open < -0.03)
    )
]

# 建立Strategy物件
example_strategy = Strategy(
    # 訊號出現後何時進出場，0為出現後馬上執行交易，1為下一個價格執行交易，2...以此類推
    next_bar=1,
    # 以甚麼價格進行交易
    trade_on='close',
    # 初始資金(後續分析中沒有使用到)
    initial_capital=1000000,
    # 交易稅
    tax_rate=0.003,
    # 交易成本(率)
    cost=0.001425
)

# 開始回測
example_strategy.run(
    # condition_in成立時欲做多的股票
    stock_2330,
    # condition_in成立時欲放空的股票
    stock_1101,
    # 進場訊號
    condition_in,
    # 出場訊號
    condition_out,
    # 對沖比率，預設為auto，亦即將兩兩欲交易的價格進行比較(trade_on)，
    # 將價格較高者部位設為1，價格較低者部位則由高價除以低價並四捨五入
    # 也可以輸入list如；[2, 1]，將會以2:1的部位進行交易(stock_to_buy:stock_to_sellshort)
    hedge_ratio='auto'
)

In [15]:
# 找出初次進場點(有進場訊號且前一期沒部位)
signal_in = example_strategy.signal[(example_strategy.signal.condition_in == True) &
                                    (lag(example_strategy.signal.stock_to_buy_position) == 0)]

In [16]:
# 由於是設定訊號出現後下個價格進場，因此在當期還不會有部位
signal_in.head()

,condition_in,condition_out,stock_to_buy_position,stock_to_sellshort_position
18,True,False,0.0,0.0
30,True,False,0.0,0.0
66,True,False,0.0,0.0
119,True,True,0.0,0.0
120,True,False,0.0,0.0


In [17]:
example_strategy.signal.iloc[18:30]

,condition_in,condition_out,stock_to_buy_position,stock_to_sellshort_position
18,True,False,0.0,0.0
19,False,False,1000.0,-4000.0
20,False,False,1000.0,-4000.0
21,False,False,1000.0,-4000.0
22,False,False,1000.0,-4000.0
23,False,False,1000.0,-4000.0
24,False,False,1000.0,-4000.0
25,True,False,1000.0,-4000.0
26,False,False,1000.0,-4000.0
27,False,False,1000.0,-4000.0


進場訊號須符合：

stock_2330.close > lag(maximum(stock_2330.high, 10))

且

stock_2330.close > stock_2330.open

In [18]:
stock_2330['lag_maximum_high_10'] = lag(maximum(stock_2330.high, 10))
stock_2330.iloc[18]

id                                    2330
name                                   台積電
date                   2001-01-05 00:00:00
open                                 21.22
high                                 22.59
low                                  21.22
close                                22.59
volume                               57537
foreign                              21355
trust                                 -276
dealer                                1559
ma20                                   NaN
ma5                                  20.87
lag_maximum_high_10                  21.47
Name: 18, dtype: object

出場訊號須符合：

(crossunder(stock_2330.ma20, stock_2330.ma5))

或

((stock_2330.close - stock_2330.open)/stock_2330.open < -0.03)

In [19]:
# 可以發現條件一未符合，但條件二有符合
stock_2330['kbar_return'] = (stock_2330.close - stock_2330.open)/stock_2330.open
stock_2330.iloc[27:29]

,id,name,date,open,high,low,close,volume,foreign,trust,dealer,ma20,ma5,lag_maximum_high_10,kbar_return
27,2330,台積電,2001-01-18,25.71,25.84,25.21,25.46,66857,33564,-647,-1568,21.8055,24.588,25.71,-0.009724
28,2330,台積電,2001-01-29,24.96,25.34,24.21,24.21,46543,7706,-1243,-1696,22.0300,24.736,25.84,-0.030048


In [20]:
# 再找一個例子
signal_in.tail()

,condition_in,condition_out,stock_to_buy_position,stock_to_sellshort_position
4330,True,False,0.0,0.0
4347,True,False,0.0,0.0
4351,True,False,0.0,0.0
4404,True,False,0.0,0.0
4450,True,True,0.0,0.0


In [21]:
example_strategy.signal.iloc[4404:4432]

,condition_in,condition_out,stock_to_buy_position,stock_to_sellshort_position
4404,True,False,0.0,0.0
4405,False,False,1000.0,-7000.0
4406,False,False,1000.0,-7000.0
4407,False,False,1000.0,-7000.0
4408,False,False,1000.0,-7000.0
4409,False,False,1000.0,-7000.0
4410,False,False,1000.0,-7000.0
4411,False,False,1000.0,-7000.0
4412,False,False,1000.0,-7000.0
4413,False,False,1000.0,-7000.0


進場訊號須符合：

stock_2330.close > lag(maximum(stock_2330.high, 10))

且

stock_2330.close > stock_2330.open

In [22]:
stock_2330.iloc[4404]

id                                    2330
name                                   台積電
date                   2018-09-27 00:00:00
open                                   264
high                                   266
low                                    262
close                                  265
volume                               38495
foreign                              14343
trust                                  150
dealer                                -367
ma20                                 260.4
ma5                                  262.7
lag_maximum_high_10                    264
kbar_return                     0.00378788
Name: 4404, dtype: object

出場訊號須符合：

(crossunder(stock_2330.ma20, stock_2330.ma5))

或

((stock_2330.close - stock_2330.open)/stock_2330.open < -0.03)

In [23]:
# 可以發現條件一未符合，但條件二有符合
stock_2330.iloc[4429:4431]

,id,name,date,open,high,low,close,volume,foreign,trust,dealer,ma20,ma5,lag_maximum_high_10,kbar_return
4429,2330,台積電,2018-11-02,236.5,236.5,233.5,236.5,30785,-1086,154,126,233.45,230.3,238.0,0.000000
4430,2330,台積電,2018-11-05,233.0,235.5,232.0,235.0,28157,1524,-20,-1383,232.70,232.8,238.0,0.008584


# analysis.py

In [24]:
from pair_trading.analysis import Analysis

# 建立分析物件
example_strategy_analysis = Analysis(
    # 欲分析的策略
    example_strategy
)

# 進行分析
example_strategy_analysis.run()

In [25]:
# 隨機找筆交易
example_strategy_analysis.stock_to_buy_trade_result.iloc[66]

entry_date                     2010-12-03 00:00:00
exit_date                      2011-03-30 00:00:00
holding_date                                   117
position_size                                 1000
entry_price                                  52.31
exit_price                                   53.84
gross_profit                                  1530
gross_return                             0.0292487
trade_cost                                 312.784
net_profit                                 1217.22
net_return                               0.0232693
maximum_favorable_excursion                7117.22
maximum_adverse_excursion                 -1002.78
Name: 66, dtype: object

In [26]:
stock_2330[stock_2330.date == '2010-12-03']

,id,name,date,open,high,low,close,volume,foreign,trust,dealer,ma20,ma5,lag_maximum_high_10,kbar_return
2478,2330,台積電,2010-12-03,51.16,53.61,51.08,52.31,130917,27078,2849,1169,49.146,50.056,50.85,0.022478


In [27]:
stock_2330[stock_2330.date == '2011-03-30']

,id,name,date,open,high,low,close,volume,foreign,trust,dealer,ma20,ma5,lag_maximum_high_10,kbar_return
2554,2330,台積電,2011-03-30,53.84,54.22,53.61,53.84,60975,-15035,540,404,53.5485,54.024,54.53,0.0


In [28]:
example_strategy_analysis.stock_to_sellshort_trade_result.iloc[66]

entry_date                     2010-12-03 00:00:00
exit_date                      2011-03-30 00:00:00
holding_date                                   117
position_size                                -3000
entry_price                                  19.95
exit_price                                    22.4
gross_profit                                 -7350
gross_return                             -0.122807
trade_cost                                 382.646
net_profit                                -7732.65
net_return                                 -0.1292
maximum_favorable_excursion                4657.35
maximum_adverse_excursion                 -7732.65
Name: 66, dtype: object

In [29]:
# 預設對沖比例為auto，進場價比例值，因此放空台泥3張
stock_2330[stock_2330.date == '2010-12-03'].close/stock_1101[stock_1101.date == '2010-12-03'].close

2478    2.622055
Name: close, dtype: float64